# **Projek Akhir Praktikum Data Science**

> Analisis Sentimen Mengenai Vaksin COVID-19 Di Indonesia Menggunakan Metode Naive Bayes Classifier dan NLP Pada Sosial Media Twitter

**Oleh Kelompok 2 :**
1. Hazlan Muhammad Qodri (123190080) @hzlnqodrey
2. Elisia Dwi Rahayu (123190062) @elisiadwirahayu
3. Shania Septika Inayasari (123190055) @shaniainayasari

**Penjelasan Projek :**

Adapun pada penelitian menekankan kepada sentimen masyarakat terhadap mengenai vaksin COVID-19. Proses analisisnya akan dilakukan berdasarkan tweet yang menyertakan tagar vaksin dan pencarian di twitter dengan keyword vaksin covid 19.

## **1. Scraping Data from Twitter**

In [ ]:
%pip install snscrape

In [ ]:
import pandas as pd
import numpy as np
import snscrape.modules.twitter as sntwitter
import re


#### Query

In [ ]:
# Get All Covid Sentiment Data from January 1st, 2020 until November 1st, 2022
query = "covid since:2020-01-01 until:2022-11-01 lang:id"
limit = 2000 # limit 50k rows

In [ ]:
tweets = []

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    else:
        tweets.append([
            tweet.date,
            tweet.username,
            tweet.content
        ])

filename = 'tweets_covid_dataset_2k_raw_noindex.csv'
tweets_df = pd.DataFrame(tweets, columns=['Tanggal', 'Username', 'Text'])
tweets_df.to_csv(filename, index=False)
print('Scraping has completed!')

## **2. Wrangling Data** (Preprocessing)

In [ ]:
%pip install tweet-preprocessor
%pip install textblob
%pip install wordcloud
%pip install nltk

In [ ]:
import preprocessor as preproc
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import csv
import string

In [303]:
# get data from dataset
data = pd.read_csv('https://raw.githubusercontent.com/hzlnqodrey/projek-akhir-prak-ds-sentimen-analisis-twitter-covid19-nlp-bayes/main/tweets_covid_dataset_2k_raw_noindex.csv')

In [ ]:
data.info()

In [ ]:
data.sample(n=5)

##### 1. Case Folding

In [304]:
data['Text'] = data['Text'].str.lower()

In [ ]:
print('Case Folding Result : \n')
data.sample(n=5)

In [ ]:
data.isnull().sum()

##### 2. Cleaning

In [305]:
# cleaning overall
def preprocessing_data(x):
    return preproc.clean(x)

data['Text'] = data['Text'].apply(preprocessing_data)


In [ ]:
print('Cleaning Result : \n')
data.sample(n=5)

In [ ]:
data['Text'].head(5)

In [306]:
# cleaning remove_comments_special
def remove_comments_special(text):
    # remove tab, new line, and back slice
    text = text.replace('\\t', " ").replace('\\n', " ").replace(
        '\\u', " ").replace('\\', " ").replace('.', " ")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(
        re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", " ", text).split())
    # remove ascii decoded
    text = ' '.join(
        re.sub("amp; ", " ", text).split())
    text = ' '.join(
        re.sub("lt; ", " ", text).split())
    text = ' '.join(
        re.sub("gt; ", " ", text).split())
    return text

data['Text'] = data['Text'].apply(remove_comments_special)

# remove symbol
def remove_symbol(text):
    text = ''.join(
        re.sub(r"[\!\@\#\$\%\^\&\*\(\)\?\,\"\|\:]+", "", text)
    )
    return text

data['Text'] = data['Text'].apply(remove_symbol)

# remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("", "", string.punctuation))

print('Cleaning Result : \n')
data.sample(n=5)

Cleaning Result : 



,Tanggal,Username,Text
771,2022-10-31 12:03:49+00:00,miff_sucii,ya betul turun jumlah per kapita juga turun banyak yg md korban covid
514,2022-10-31 14:09:37+00:00,Kochikecil,hukum mati tidak bisa berantas korupsi
1369,2022-10-31 08:01:54+00:00,jia_shahril,ye wehh samaa aku sebelum covid memng dah selalu lupa tpi lepas covid lagilah teruk penyakit lupa aku
1851,2022-10-31 05:07:12+00:00,BuletinTV3,covid-19 | pemakaian pelitup muka kekal secara sukarela tetapi sangat digalakkan di kawasan tertutup dan sesak tegas menteri kesihatan khairy jamaluddin video penuh:
580,2022-10-31 13:30:15+00:00,AyoGerakBareng,ada begitu banyak yang berubah ketika pandemi covid-19 melanda dunia mulai dari perubahan sosial interaksi publik hingga kegiatan-kegiatan keagamaan termasuk peribadatan umat islam virus corona ramadhan dan perubahan tradisi selama ramadhan


In [ ]:
data.head(50)

##### 3. Tokenizing

In [307]:
def tokenize_data(text):
    return word_tokenize(text)

data['Text_Clean'] = data['Text'].apply(tokenize_data)

In [308]:
print('Tokenizing Result : \n')
data.sample(n=5)

Tokenizing Result : 



,Tanggal,Username,Text,Text_Clean
302,2022-10-31 16:04:50+00:00,estiyunho_,hahaha pindaaaah tapi kapan lalu masih kesana kok sebelum covid pas hari raya di sana helm masih sering ilang kak,"[hahaha, pindaaaah, tapi, kapan, lalu, masih, kesana, kok, sebelum, covid, pas, hari, raya, di, sana, helm, masih, sering, ilang, kak]"
43,2022-10-31 23:19:40+00:00,pemudalincah,ini di kantor pada rame covid lg tbtb org kena,"[ini, di, kantor, pada, rame, covid, lg, tbtb, org, kena]"
373,2022-10-31 15:18:11+00:00,br02vlogs,mod g china me new virus brackout hua hn jo covid se gunah ghatak hn,"[mod, g, china, me, new, virus, brackout, hua, hn, jo, covid, se, gunah, ghatak, hn]"
1054,2022-10-31 10:09:32+00:00,Mr_Hendra_H,video sandiwara merebaknya covid di wuhan semakin menguatkan sinyalemen bapak bahwa pemilu pilpres akan ditunda mobilitas warga akan semakin dikekang dan jokowi akan melenggang mulus tiga periode,"[video, sandiwara, merebaknya, covid, di, wuhan, semakin, menguatkan, sinyalemen, bapak, bahwa, pemilu, pilpres, akan, ditunda, mobilitas, warga, akan, semakin, dikekang, dan, jokowi, akan, melenggang, mulus, tiga, periode]"
420,2022-10-31 14:52:57+00:00,atiqah_tykass,aku putus cinta cuti seminggu covid,"[aku, putus, cinta, cuti, seminggu, covid]"


##### 4. Filtering

In [309]:
# Filtering | Singkatan Indo
normalizad_word = pd.read_csv(
    "https://raw.githubusercontent.com/meisaputri21/Indonesian-Twitter-Emotion-Dataset/master/kamus_singkatan.csv", sep=";", header=None)
normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]


def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]


data['Text_Clean'] = data['Text_Clean'].apply(normalized_term)
data.head(5)

,Tanggal,Username,Text,Text_Clean
0,2022-10-31 23:59:58+00:00,lordkuyang,dulu di barat upn ada warung kayuh bambai bumbu habangnya sungguh banjar banar semenjak tahun ke covid mereka tutup padahal itu warung yang sangat mengobati kerinduan sama masakan banjar nasi kuning dan soto banjarnya juara sekali,"[dulu, di, barat, upn, ada, warung, kayuh, bambai, bumbu, habangnya, sungguh, banjar, banar, semenjak, tahun, ke, covid, mereka, tutup, padahal, itu, warung, yang, sangat, mengobati, kerinduan, sama, masakan, banjar, nasi, kuning, dan, soto, banjarnya, juara, sekali]"
1,2022-10-31 23:58:49+00:00,s_h_y_l_l_a,segala sakit yg disalahin vaksin covid lieur,"[segala, sakit, yang , disalahin, vaksin, covid, lieur]"
2,2022-10-31 23:58:14+00:00,kunsh72,bukankah kamu dulu bagian rezim namun setelah terdepak dan sakit hati karena rezekinya hilang maka jadi sok kritis dulu mengkritisi vaksin covid namun saat divaksin pamer di medsos ucapanmu tidak konsisten bro semua karena sakit hati atas hilangnya rezeki,"[bukankah, kamu, dulu, bagian, rezim, namun, setelah, terdepak, dan, sakit, hati, karena, rezekinya, hilang, maka, jadi, sok, kritis, dulu, mengkritisi, vaksin, covid, namun, saat, divaksin, pamer, di, medsos, ucapanmu, tidak, konsisten, bro, semua, karena, sakit, hati, atas, hilangnya, rezeki]"
3,2022-10-31 23:57:58+00:00,erni076,aku kan ganti lama dulu sebelum covid menyerang sdh lama beda tahun y beda hrga mas,"[saya, kan, ganti, lama, dulu, sebelum, covid, menyerang, sudah , lama, beda, tahun, y, beda, hrga, mas]"
4,2022-10-31 23:57:38+00:00,KENTUSIAS,trus masku kena covid lagi kek yah udah endemi masi ae,"[terus , masku, kena, covid, lagi, kek, yah, sudah , endemi, masi, ae]"


In [312]:
# Filtering | Stop Word
list_stopwords = (['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'seperti', 'jika', 'jika', 'sehingga', 'mungkin', 'kembali', 'dan', 'ini', 'karena', 'oleh', 'saat', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'sebagai', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bahwa', 'atau', 'dengan', 'akan', 'juga', 'kalau', 'ada', 'terhadap', 'secara', 'agar', 'lain', 'jadi', 'yang ', 'sudah', 'sudah begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi',
                  'sementara', 'tetapi', 'apakah', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'juga', 'mari', 'nanti', 'melainkan', 'oh', 'ok', 'sebetulnya', 'setiap', 'sesuatu', 'pasti', 'saja', 'toh', 'ya', 'walau', 'apalagi', 'bagaimanapun', 'yg', 'dg', 'rt', 'dgn', 'ny', 'd', 'klo', 'kalo', 'amp', 'biar', 'bikin', 'bilang', 'krn', 'nya', 'nih', 'sih', 'ah', 'ssh', 'om', 'ah', 'si', 'tau', 'tuh', 'utk', 'ya', 'cek', 'jd', 'aja', 't', 'nyg', 'hehe', 'pen', 'nan', 'loh', 'rt', '&amp', 'yah', 'ni', 'ret', 'za', 'nak', 'haa', 'zaa', 'maa', 'lg', 'eh', 'hmm', 'kali'])

list_stopwords = set(list_stopwords)


def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]


data['Text_Clean'] = data['Text_Clean'].apply(stopwords_removal)
data.head(5)

,Tanggal,Username,Text,Text_Clean
0,2022-10-31 23:59:58+00:00,lordkuyang,dulu di barat upn ada warung kayuh bambai bumbu habangnya sungguh banjar banar semenjak tahun ke covid mereka tutup padahal itu warung yang sangat mengobati kerinduan sama masakan banjar nasi kuning dan soto banjarnya juara sekali,"[dulu, barat, upn, warung, kayuh, bambai, bumbu, habangnya, sungguh, banjar, banar, semenjak, tahun, covid, mereka, tutup, padahal, warung, sangat, mengobati, kerinduan, sama, masakan, banjar, nasi, kuning, soto, banjarnya, juara, sekali]"
1,2022-10-31 23:58:49+00:00,s_h_y_l_l_a,segala sakit yg disalahin vaksin covid lieur,"[segala, sakit, disalahin, vaksin, covid, lieur]"
2,2022-10-31 23:58:14+00:00,kunsh72,bukankah kamu dulu bagian rezim namun setelah terdepak dan sakit hati karena rezekinya hilang maka jadi sok kritis dulu mengkritisi vaksin covid namun saat divaksin pamer di medsos ucapanmu tidak konsisten bro semua karena sakit hati atas hilangnya rezeki,"[bukankah, kamu, dulu, bagian, rezim, setelah, terdepak, sakit, hati, rezekinya, hilang, sok, kritis, dulu, mengkritisi, vaksin, covid, divaksin, pamer, medsos, ucapanmu, tidak, konsisten, bro, semua, sakit, hati, atas, hilangnya, rezeki]"
3,2022-10-31 23:57:58+00:00,erni076,aku kan ganti lama dulu sebelum covid menyerang sdh lama beda tahun y beda hrga mas,"[saya, kan, ganti, lama, dulu, covid, menyerang, sudah , lama, beda, tahun, y, beda, hrga, mas]"
4,2022-10-31 23:57:38+00:00,KENTUSIAS,trus masku kena covid lagi kek yah udah endemi masi ae,"[terus , masku, kena, covid, kek, sudah , endemi, masi, ae]"


## **2.1 Menejermah Tweet Clean hasil Preproc ke Bahasa Inggris** (Translating)

In [ ]:
%pip install googletrans==4.0.0rc1

In [ ]:
from googletrans import Translator

translate = Translator()

out = translate.translate(['veritas lux mea', 'lorem ipsum', 'gnocchi', 'dolor sit amet'], dest='en')

print(out.text)

## **2.2 Stemming**

## **3. Modelling Data** (Modeling and Training)

## **4. Klasifikasi Data dengan Naive Bayes** (Classification)